In [10]:
import tensorflow as tf
import numpy as np

In [2]:
y_true = tf.constant([[0., 1.], [0., 0.]])
y_pred = tf.constant([[1., 1.], [1., 0.]])

2022-09-09 20:38:32.737227: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
loss_val = (tf.math.log(y_true + 1) - tf.math.log(y_pred + 1)) ** 2
print(tf.reduce_sum(loss_val) / 4)

tf.Tensor(0.2402265, shape=(), dtype=float32)


In [20]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()
msle(y_true, y_pred).numpy()

0.24022643

In [41]:
a_loss = loss_val.numpy()[0,0]
a_loss = 0.13

diff = np.e ** np.sqrt(a_loss)
print(diff)

1.4341253160944203


In [39]:
msle([10000.], [11000.]).numpy()

0.009082219

## Keras

In [4]:
import tensorflow as tf

2022-12-11 21:07:22.298228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
x = tf.keras.layers.Input(shape=(2,), dtype=tf.float32)
y = x

model = tf.keras.models.Model(inputs={'x': x}, outputs={'y': y})

2022-12-11 21:07:27.687850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.inputs

[<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'input_1')>]

In [7]:
model.input

{'x': <KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'input_1')>}

In [11]:
model.input['x'].dtype

tf.float32

In [13]:
model.input['x'].shape[1:].shape

AttributeError: 'TensorShape' object has no attribute 'shape'